fmriprep calculates a confounds.tsv-file per echo

Here, we re-create a single confounds.tsv-file after recombination of data


#### This needs to be run in T1w-space, so it requires that the echo combination is also done in T1w-space

In [1]:
from fmriprep.workflows.bold.confounds import init_bold_confs_wf
from fmriprep.utils.bids import collect_data, get_second_inversion
from fmriprep.utils.misc import fix_multi_T1w_source_name
import numpy as np

/usr/local/miniconda/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/usr/local/miniconda/lib/python3.6/site-packages/bids/grabbids/__init__.py:6: FutureWarning: grabbids has been renamed to layout in version 0.6.5, and will be removed in version 0.8
  warnings.warn("grabbids has been renamed to layout in version 0.6.5, and will be removed in version 0.8", FutureWarning)


In [2]:
from nipype.pipeline import engine as pe
from nipype.interfaces import utility as niu, fsl
from nipype.interfaces.nilearn import SignalExtraction
from nipype.algorithms import confounds as nac

from niworkflows.data import get_mni_icbm152_linear, get_mni_icbm152_nlin_asym_09c
from niworkflows.interfaces.segmentation import ICA_AROMARPT
from niworkflows.interfaces.masks import ROIsPlot
from niworkflows.interfaces.fixes import FixHeaderApplyTransforms as ApplyTransforms

from fmriprep.engine import Workflow
from fmriprep.interfaces import (
    TPM2ROI, AddTPMs, AddTSVHeader, GatherConfounds, ICAConfounds,
    FMRISummary, DerivativesDataSink
)
from fmriprep.interfaces.patches import (
    RobustACompCor as ACompCor,
    RobustTCompCor as TCompCor
)

from fmriprep.workflows.anatomical import register_func
from fmriprep.workflows.bold.resampling import init_bold_mni_trans_wf
DEFAULT_MEMORY_MIN_GB = 0.01


def init_bold_confs_wf(mem_gb, metadata, name="bold_confs_wf"):
    """
    This workflow calculates confounds for a BOLD series, and aggregates them
    into a :abbr:`TSV (tab-separated value)` file, for use as nuisance
    regressors in a :abbr:`GLM (general linear model)`.

    The following confounds are calculated, with column headings in parentheses:

    #. Region-wise average signal (``CSF``, ``WhiteMatter``, ``GlobalSignal``)
    #. DVARS - standard, nonstandard, and voxel-wise standard variants
       (``stdDVARS``, ``non-stdDVARS``, ``vx-wisestdDVARS``)
    #. Framewise displacement, based on MCFLIRT motion parameters
       (``FramewiseDisplacement``)
    #. Temporal CompCor (``tCompCorXX``)
    #. Anatomical CompCor (``aCompCorXX``)
    #. Cosine basis set for high-pass filtering w/ 0.008 Hz cut-off
       (``CosineXX``)
    #. Non-steady-state volumes (``NonSteadyStateXX``)
    #. Estimated head-motion parameters, in mm and rad
       (``X``, ``Y``, ``Z``, ``RotX``, ``RotY``, ``RotZ``)


    Prior to estimating aCompCor and tCompCor, non-steady-state volumes are
    censored and high-pass filtered using a :abbr:`DCT (discrete cosine
    transform)` basis.
    The cosine basis, as well as one regressor per censored volume, are included
    for convenience.

    .. workflow::
        :graph2use: orig
        :simple_form: yes

        from fmriprep.workflows.bold.confounds import init_bold_confs_wf
        wf = init_bold_confs_wf(
            mem_gb=1,
            metadata={})

    **Parameters**

        mem_gb : float
            Size of BOLD file in GB - please note that this size
            should be calculated after resamplings that may extend
            the FoV
        metadata : dict
            BIDS metadata for BOLD file
        name : str
            Name of workflow (default: ``bold_confs_wf``)

    **Inputs**

        bold
            BOLD image, after the prescribed corrections (STC, HMC and SDC)
            when available.
        bold_mask
            BOLD series mask
        movpar_file
            SPM-formatted motion parameters file
        t1_mask
            Mask of the skull-stripped template image
        t1_tpms
            List of tissue probability maps in T1w space
        t1_bold_xform
            Affine matrix that maps the T1w space into alignment with
            the native BOLD space

    **Outputs**

        confounds_file
            TSV of all aggregated confounds
        rois_report
            Reportlet visualizing white-matter/CSF mask used for aCompCor,
            the ROI for tCompCor and the BOLD brain mask.

    """
    workflow = Workflow(name=name)
    workflow.__desc__ = """\
Several confounding time-series were calculated based on the
*preprocessed BOLD*: framewise displacement (FD), DVARS and
three region-wise global signals.
FD and DVARS are calculated for each functional run, both using their
implementations in *Nipype* [following the definitions by @power_fd_dvars].
The three global signals are extracted within the CSF, the WM, and
the whole-brain masks.
Additionally, a set of physiological regressors were extracted to
allow for component-based noise correction [*CompCor*, @compcor].
Principal components are estimated after high-pass filtering the
*preprocessed BOLD* time-series (using a discrete cosine filter with
128s cut-off) for the two *CompCor* variants: temporal (tCompCor)
and anatomical (aCompCor).
Six tCompCor components are then calculated from the top 5% variable
voxels within a mask covering the subcortical regions.
This subcortical mask is obtained by heavily eroding the brain mask,
which ensures it does not include cortical GM regions.
For aCompCor, six components are calculated within the intersection of
the aforementioned mask and the union of CSF and WM masks calculated
in T1w space, after their projection to the native space of each
functional run (using the inverse BOLD-to-T1w transformation).
The head-motion estimates calculated in the correction step were also
placed within the corresponding confounds file.
"""
    inputnode = pe.Node(niu.IdentityInterface(
        fields=['bold', 'bold_mask', 'movpar_file', 
                't1_mask', 
                't1_tpms']),
        name='inputnode')
    outputnode = pe.Node(niu.IdentityInterface(
        fields=['confounds_file']),
        name='outputnode')

    # downsample t1w-space images
    downsample_tpms1 = pe.MapNode(
        niu.Function(input_names=['source_img', 'target_img', 'run_rigid', 'run_syn', 'run_affine'],
                     output_names=['transformed_source', 'mapping', 'inverse', 'mapping_cmap',
                                   'inverse_cmap', 'out_report'],
                     function=register_func), iterfield=['source_img'],
        name='downsample_tmps')
    downsample_tpms1.inputs.run_rigid = False  # all False: only apply affine
    downsample_tpms1.inputs.run_syn = False
    downsample_tpms1.inputs.run_affine = False
    
#     downsample_tpms2 = pe.Node(
#         niu.Function(input_names=['source_img', 'target_img', 'run_rigid', 'run_syn', 'run_affine'],
#                      output_names=['transformed_source', 'mapping', 'inverse', 'mapping_cmap',
#                                    'inverse_cmap', 'out_report'],
#                      function=register_func), 
#         name='downsample_tmps2')
#     downsample_tpms2.inputs.run_rigid = False  # all False: only apply affine
#     downsample_tpms2.inputs.run_syn = False
#     downsample_tpms2.inputs.run_affine = False
    
#     downsample_tpms1 = pe.Node(
#         niu.Function(input_names=['source_img', 'target_img', 'run_rigid', 'run_syn', 'run_affine'],
#                      output_names=['transformed_source', 'mapping', 'inverse', 'mapping_cmap',
#                                    'inverse_cmap', 'out_report'],
#                      function=register_func), 
#         name='downsample_tmps3')
#     downsample_tpms3.inputs.run_rigid = False  # all False: only apply affine
#     downsample_tpms3.inputs.run_syn = False
#     downsample_tpms3.inputs.run_affine = False
    
    # get single slice for target of nighres registration
    def get_first_slice(fn):
        import nibabel as nib
        hdr = nib.load(fn)
        dat = hdr.get_data()[:,:,:,0]
        
        out_fn = fn.replace('.nii.gz', '_first_slice.nii.gz')
        nib.save(nib.Nifti1Image(dat, hdr.affine), out_fn)
        return out_fn
    
    first_slice_getter = pe.Node(niu.Function(input_names=['fn'],
                                              output_names=['out_fn'],
                                             function = get_first_slice),
                                name='get_first_slice')
    
    # Get masks ready in T1w space
    acc_tpm = pe.Node(AddTPMs(indices=[0, 2]), name='tpms_add_csf_wm')  # acc stands for aCompCor
    csf_roi = pe.Node(TPM2ROI(erode_mm=0, mask_erode_mm=30), name='csf_roi')
    wm_roi = pe.Node(TPM2ROI(
        erode_prop=0.6, mask_erode_prop=0.6**3),  # 0.6 = radius; 0.6^3 = volume
        name='wm_roi')
    acc_roi = pe.Node(TPM2ROI(
        erode_prop=0.6, mask_erode_prop=0.6**3),  # 0.6 = radius; 0.6^3 = volume
        name='acc_roi')

    # Map ROIs in T1w space into BOLD space
    # BOLD was already transformed to T1w space, so no need for this
#     csf_tfm = pe.Node(ApplyTransforms(interpolation='NearestNeighbor', float=True),
#                       name='csf_tfm', mem_gb=0.1)
#     wm_tfm = pe.Node(ApplyTransforms(interpolation='NearestNeighbor', float=True),
#                      name='wm_tfm', mem_gb=0.1)
#     acc_tfm = pe.Node(ApplyTransforms(interpolation='NearestNeighbor', float=True),
#                       name='acc_tfm', mem_gb=0.1)
#     tcc_tfm = pe.Node(ApplyTransforms(interpolation='NearestNeighbor', float=True),
#                       name='tcc_tfm', mem_gb=0.1)

    ###### SM: ants_coreg gives *two* transforms, only *one* needs to be inverted
    def _get_invert_transform_flags(xform):
        if isinstance(xform, list):
            if len(xform) == 2:
                print(xform)
                print('Setting invert transform flags to [True, True]')
                # the order here should be [ants_coreg.mat, affine.txt].
                # note that you would expect only ants_coreg.mat to be inverted,
                # but some plotting showed *both* require inversion. I don't know why, but
                # the output fits only if both are inverted.
                return [True, True]
        else:
            return [False]

    # Ensure ROIs don't go off-limits (reduced FoV)
    csf_msk = pe.Node(niu.Function(function=_maskroi), name='csf_msk')
    wm_msk = pe.Node(niu.Function(function=_maskroi), name='wm_msk')
    acc_msk = pe.Node(niu.Function(function=_maskroi), name='acc_msk')
    tcc_msk = pe.Node(niu.Function(function=_maskroi), name='tcc_msk')

    # DVARS
    dvars = pe.Node(nac.ComputeDVARS(save_all=True, remove_zerovariance=True),
                    name="dvars", mem_gb=mem_gb)

    # Frame displacement
    fdisp = pe.Node(nac.FramewiseDisplacement(parameter_source="SPM"),
                    name="fdisp", mem_gb=mem_gb)

    # a/t-CompCor
    non_steady_state = pe.Node(nac.NonSteadyStateDetector(), name='non_steady_state')
    tcompcor = pe.Node(TCompCor(
        components_file='tcompcor.tsv', pre_filter='cosine', save_pre_filter=True,
        percentile_threshold=.05), name="tcompcor", mem_gb=mem_gb)

    acompcor = pe.Node(ACompCor(
        components_file='acompcor.tsv', pre_filter='cosine', save_pre_filter=True),
        name="acompcor", mem_gb=mem_gb)

    # Set TR if present
    if 'RepetitionTime' in metadata:
        tcompcor.inputs.repetition_time = metadata['RepetitionTime']
        acompcor.inputs.repetition_time = metadata['RepetitionTime']

    # Global and segment regressors
    mrg_lbl = pe.Node(niu.Merge(3), name='merge_rois', run_without_submitting=True)
    signals = pe.Node(SignalExtraction(class_labels=["CSF", "WhiteMatter", "GlobalSignal"]),
                      name="signals", mem_gb=mem_gb)

    # Arrange confounds
    add_header = pe.Node(AddTSVHeader(columns=["X", "Y", "Z", "RotX", "RotY", "RotZ"]),
                         name="add_header", mem_gb=0.01, run_without_submitting=True)
    concat = pe.Node(GatherConfounds(), name="concat", mem_gb=0.01, run_without_submitting=True)

    # Generate reportlet
    mrg_compcor = pe.Node(niu.Merge(2), name='merge_compcor', run_without_submitting=True)
    rois_plot = pe.Node(ROIsPlot(colors=['r', 'b', 'magenta'], generate_report=True),
                        name='rois_plot', mem_gb=mem_gb)

    ds_report_bold_rois = pe.Node(
        DerivativesDataSink(suffix='rois'),
        name='ds_report_bold_rois', run_without_submitting=True,
        mem_gb=DEFAULT_MEMORY_MIN_GB)

    def _pick_csf(files):
        return files[0]

    def _pick_wm(files):
        return files[-1]

    workflow.connect([
        # Massage ROIs (in T1w space)
        (inputnode, downsample_tpms1, [('t1_tpms', 'source_img')]),
        (inputnode, first_slice_getter, [('bold', 'fn')]),
        (first_slice_getter, downsample_tpms1, [('out_fn', 'target_img')]),
        (downsample_tpms1, acc_tpm, [('transformed_source', 'in_files')]),
        (downsample_tpms1, csf_roi, [(('transformed_source', _pick_csf), 'in_tpm')]),
        (inputnode, csf_roi, [('t1_mask', 'in_mask')]),
        (downsample_tpms1, wm_roi, [(('transformed_source', _pick_wm), 'in_tpm')]),
        (inputnode, wm_roi, [('t1_mask', 'in_mask')]),
        (inputnode, acc_roi, [('t1_mask', 'in_mask')]),
        (acc_tpm, acc_roi, [('out_file', 'in_tpm')]),
        
        # Map ROIs to BOLD
#         (inputnode, csf_tfm, [('bold_mask', 'reference_image'),
#                               ('t1_bold_xform', 'transforms'),
#                               (('t1_bold_xform', _get_invert_transform_flags), 'invert_transform_flags')]),  ### SM
        # transform CSF img
#         (csf_roi, downsample_tpms1, [('roi_file', 'source_img')]),
#         (inputnode, first_slice_getter, [('bold', 'fn')]),
#         (first_slice_getter, downsample_tpms1, [('out_fn', 'target_img')])

#         (inputnode, wm_tfm, [('bold_mask', 'reference_image'),
#                              ('t1_bold_xform', 'transforms'),
#                              (('t1_bold_xform', _get_invert_transform_flags), 'invert_transform_flags')]),  ### SM
#         (wm_roi, wm_tfm, [('roi_file', 'input_image')]),
#         (inputnode, acc_tfm, [('bold_mask', 'reference_image'),
#                               ('t1_bold_xform', 'transforms'),
#                               (('t1_bold_xform', _get_invert_transform_flags), 'invert_transform_flags')]),  ### SM
#         (acc_roi, acc_tfm, [('roi_file', 'input_image')]),
#         (inputnode, tcc_tfm, [('bold_mask', 'reference_image'),
#                               ('t1_bold_xform', 'transforms'),
#                               (('t1_bold_xform', _get_invert_transform_flags), 'invert_transform_flags')]),  ### SM
#         (csf_roi, tcc_tfm, [('eroded_mask', 'input_image')]),
        # Mask ROIs with bold_mask
        (inputnode, csf_msk, [('bold_mask', 'in_mask')]),
        (inputnode, wm_msk, [('bold_mask', 'in_mask')]),
        (inputnode, acc_msk, [('bold_mask', 'in_mask')]),
        (inputnode, tcc_msk, [('bold_mask', 'in_mask')]),
        # connect inputnode to each non-anatomical confound node
        (inputnode, dvars, [('bold', 'in_file'),
                            ('bold_mask', 'in_mask')]),
        (inputnode, fdisp, [('movpar_file', 'in_file')]),

        # Calculate nonsteady state
        (inputnode, non_steady_state, [('bold', 'in_file')]),

        # tCompCor
        (inputnode, tcompcor, [('bold', 'realigned_file')]),
        (non_steady_state, tcompcor, [('n_volumes_to_discard', 'ignore_initial_volumes')]),
#        (tcc_tfm, tcc_msk, [('output_image', 'roi_file')]),
        (csf_roi, tcc_msk, [('eroded_mask', 'roi_file')]),  # tcc uses csf' eroded mask
        (tcc_msk, tcompcor, [('out', 'mask_files')]),

        # aCompCor
        (inputnode, acompcor, [('bold', 'realigned_file')]),
        (non_steady_state, acompcor, [('n_volumes_to_discard', 'ignore_initial_volumes')]),
#       (acc_tfm, acc_msk, [('output_image', 'roi_file')]),
        (acc_roi, acc_msk, [('roi_file', 'roi_file')]),
        (acc_msk, acompcor, [('out', 'mask_files')]),

        # Global signals extraction (constrained by anatomy)
        (inputnode, signals, [('bold', 'in_file')]),
#        (csf_tfm, csf_msk, [('output_image', 'roi_file')]),
        (csf_roi, csf_msk, [('roi_file', 'roi_file')]),
        (csf_msk, mrg_lbl, [('out', 'in1')]),
#        (wm_tfm, wm_msk, [('output_image', 'roi_file')]),
        (wm_roi, wm_msk, [('roi_file', 'roi_file')]),
        (wm_msk, mrg_lbl, [('out', 'in2')]),
        (inputnode, mrg_lbl, [('bold_mask', 'in3')]),
        (mrg_lbl, signals, [('out', 'label_files')]),

        # Collate computed confounds together
        (inputnode, add_header, [('movpar_file', 'in_file')]),
        (signals, concat, [('out_file', 'signals')]),
        (dvars, concat, [('out_all', 'dvars')]),
        (fdisp, concat, [('out_file', 'fd')]),
        (tcompcor, concat, [('components_file', 'tcompcor'),
                            ('pre_filter_file', 'cos_basis')]),
        (acompcor, concat, [('components_file', 'acompcor')]),
        (add_header, concat, [('out_file', 'motion')]),

        # Set outputs
        (concat, outputnode, [('confounds_file', 'confounds_file')]),
#         (inputnode, rois_plot, [('bold', 'in_file'),
#                                 ('bold_mask', 'in_mask')]),
#         (tcompcor, mrg_compcor, [('high_variance_masks', 'in1')]),
#         (acc_msk, mrg_compcor, [('out', 'in2')]),
#         (mrg_compcor, rois_plot, [('out', 'in_rois')]),
#         (rois_plot, ds_report_bold_rois, [('out_report', 'in_file')]),
    ])

    return workflow

def _maskroi(in_mask, roi_file):
    import numpy as np
    import nibabel as nb
    from nipype.utils.filemanip import fname_presuffix

    roi = nb.load(roi_file)
    roidata = roi.get_data().astype(np.uint8)
    msk = nb.load(in_mask).get_data().astype(bool)
    roidata[~msk] = 0
    roi.set_data_dtype(np.uint8)

    out = fname_presuffix(roi_file, suffix='_boldmsk')
    roi.__class__(roidata, roi.affine, roi.header).to_filename(out)
    return out

In [ ]:
bids_dir = './data/raw/bids/'
session_id = 'me'
task_id = None
mem_gb = {'filesize': 1, 'resampled': 1, 'largemem': 1}
subject_id = 1
subject_data, layout = collect_data(bids_dir, subject_id, session_id, task_id)
fn = './data/raw/bids/sub-01/ses-me/func/sub-01_ses-me_task-stop_run-1_echo-1_bold.nii'
metadata = layout.get_metadata(fn)



# import nipype.pipeline.engine as pe
# import nipype.interfaces.utility as niu
# import nipype.interfaces.io as nio

# outputnode = pe.Node(nio.DataSink(), name='inputnode')

# templates = {'bold': '/home/neuro/work_dir/data/deriv/fmriprep/sub-{subject_id}/ses-me/func/sub-{subject_id}_ses-me_task-stop_run-{run}_echo-1_bold_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_preproc_hp.nii.gz',
#              'bold_mask': '/home/neuro/work_dir/data/deriv/fmriprep/sub-{subject_id}/ses-me/func/sub-{subject_id}_ses-me_task-stop_run-{run}_echo-1_bold_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_brainmask.nii.gz',
#              't1w_mask': '/home/neuro/work_dir/data/deriv/fmriprep/sub-{subject_id}/ses-me/func/sub-{subject_id}_ses-me_task-stop_run-{run}_echo-1_bold_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_brainmask.nii.gz',
             
#              't1_csf': '/home/neuro/work_dir/data/deriv/fmriprep/sub-{subject_id}/anat/sub-{subject_id}_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_class-CSF_probtissue.nii.gz',
#              't1_gm': '/home/neuro/work_dir/data/deriv/fmriprep/sub-{subject_id}/anat/sub-{subject_id}_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_class-GM_probtissue.nii.gz',
#              't1_wm': '/home/neuro/work_dir/data/deriv/fmriprep/sub-{subject_id}/anat/sub-{subject_id}_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_class-WM_probtissue.nii.gz',
# #                          '/home/neuro/work_dir/data/deriv/fmriprep/sub-{subject_id}/anat/sub-{subject_id}_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_class-GM_probtissue.nii.gz',
# #                          '/home/neuro/work_dir/data/deriv/fmriprep/sub-{subject_id}/anat/sub-{subject_id}_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_class-WM_probtissue.nii.gz'],
#              'mov_par': '/home/neuro/work_dir/workflow_folders/fmriprep_wf*/single_subject_{subject_id}_wf/func_preproc_ses_me_task_stop_run_{run}_echo_1_wf/bold_hmc_wf/normalize_motion/motion_params.txt'
#             }

# selector = pe.Node(nio.SelectFiles(templates), name='selector')
# selector.iterables = [('subject_id', [str(x).zfill(2) for x in range(1,19) if not x==12]),
#                       ('run', [1,2,3])]
# merge_tpms = pe.Node(niu.Merge(3), name='tpm_merger')

# bold_confs_wf = init_bold_confs_wf(mem_gb=mem_gb, metadata=metadata)
# bold_confs_wf.base_dir = '/home/neuro/work_dir/workflow_folders/confounds_me_sub-01_run-1'

# #         bold_confs_wf.inputs.inputnode.movpar_file = movpar_file
# #         bold_confs_wf.inputs.inputnode.t1_mask = t1_mask
# #         bold_confs_wf.inputs.inputnode.t1_tpms = t1_tpms

# bold_confs_wf.connect([selector, bold_confs_wf.inputs.inputnode, [('bold', 'bold'),
#                                                                   ('bold_mask', 'bold_mask'),
#                                                                   ('t1w_mask', 't1_mask'),
#                                                                   ('mov_par', 'mov_par')],
#                       selector, merge_tpms, [('t1_csf', 'in1'),
#                                              ('t1_gm', 'in2'),
#                                              ('t1_wm', 'in3')],
#                       merge_tpms, bold_confs_wf.inputs.inputnode, [('out', 't1_tpms')]])
#                       ('t1_tpms', 't1_tpms')])

# # loop over subbs

for subject_id in np.arange(1,19):
    if subject_id == 12:
        continue
    
    subject_data, layout = collect_data(bids_dir, subject_id, session_id, task_id)
    
    for run in [1,2,3]:

        # get fn for metadata. NOT the fn used to get confounds from
        fn = './data/raw/bids/sub-%s/ses-me/func/sub-%s_ses-me_task-stop_run-%d_echo-1_bold.nii' %(str(subject_id).zfill(2), str(subject_id).zfill(2),run)
        metadata = layout.get_metadata(fn)
        bold_confs_wf = init_bold_confs_wf(mem_gb=mem_gb, metadata=metadata)
        bold_confs_wf.base_dir = '/home/neuro/work_dir/workflow_folders/confounds_me_sub-%s_run-%d' %(str(subject_id).zfill(2),run)

        # # inputs
        bold = '/home/neuro/work_dir/data/deriv/fmriprep/sub-%s/ses-me/func/sub-%s_ses-me_task-stop_run-%d_echo-1_bold_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_preproc_hp.nii.gz' %(str(subject_id).zfill(2), str(subject_id).zfill(2),run)
        bold_mask = bold.replace('_preproc_hp.nii.gz', '_brainmask.nii.gz').replace('optcomb', 'echo-1')
        if subject_id < 5:
            movpar_file = '/home/neuro/work_dir/workflow_folders/fmriprep_wf_sub-01-04/single_subject_%s_wf/func_preproc_ses_me_task_stop_run_%d_echo_1_wf/bold_hmc_wf/normalize_motion/motion_params.txt' %(str(subject_id).zfill(2), run)
        else:
            movpar_file = '/home/neuro/work_dir/workflow_folders/fmriprep_wf/single_subject_%s_wf/func_preproc_ses_me_task_stop_run_%d_echo_1_wf/bold_hmc_wf/normalize_motion/motion_params.txt' %(str(subject_id).zfill(2), run)

        t1_mask = bold_mask #'/home/neuro/work_dir/data/deriv/fmriprep/sub-%s/anat/sub-%s_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_brainmask.nii.gz' %(str(subject_id).zfill(2), str(subject_id).zfill(2))
        t1_tpms = [
            '/home/neuro/work_dir/data/deriv/fmriprep/sub-%s/anat/sub-%s_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_class-CSF_probtissue.nii.gz'%(str(subject_id).zfill(2), str(subject_id).zfill(2)), 
            '/home/neuro/work_dir/data/deriv/fmriprep/sub-%s/anat/sub-%s_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_class-GM_probtissue.nii.gz'%(str(subject_id).zfill(2), str(subject_id).zfill(2)),
            '/home/neuro/work_dir/data/deriv/fmriprep/sub-%s/anat/sub-%s_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_class-WM_probtissue.nii.gz'%(str(subject_id).zfill(2), str(subject_id).zfill(2))
        ]

        bold_confs_wf.inputs.inputnode.bold = bold
        bold_confs_wf.inputs.inputnode.bold_mask = bold_mask
        bold_confs_wf.inputs.inputnode.movpar_file = movpar_file
        bold_confs_wf.inputs.inputnode.t1_mask = t1_mask
        bold_confs_wf.inputs.inputnode.t1_tpms = t1_tpms

        bold_confs_wf.outputs.outputnode.confounds_file = '/home/neuro/work_dir/confounds_me/sub-%s_ses-me_task-stop_run-%d_optcomb_bold_space-T1w_confounds.tsv' %(str(subject_id).zfill(2),run)
        
        bold_confs_wf.run()

181217-15:49:54,848 nipype.workflow INFO:
	 Workflow bold_confs_wf settings: ['check', 'execution', 'logging', 'monitoring']
181217-15:49:54,864 nipype.workflow INFO:
	 Running serially.
181217-15:49:54,866 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.add_header" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-01_run-1/bold_confs_wf/add_header".
181217-15:49:54,894 nipype.workflow INFO:
	 [Node] Cached "bold_confs_wf.add_header" - collecting precomputed outputs
181217-15:49:54,896 nipype.workflow INFO:
	 [Node] "bold_confs_wf.add_header" found cached.
181217-15:49:54,898 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.non_steady_state" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-01_run-1/bold_confs_wf/non_steady_state".
181217-15:49:54,916 nipype.workflow INFO:
	 [Node] Cached "bold_confs_wf.non_steady_state" - collecting precomputed outputs
181217-15:49:54,918 nipype.workflow INFO:
	 [Node] "bold_confs_wf.non_steady_state" found cached.
1

181217-15:49:55,219 nipype.workflow INFO:
	 [Node] Cached "bold_confs_wf.non_steady_state" - collecting precomputed outputs
181217-15:49:55,220 nipype.workflow INFO:
	 [Node] "bold_confs_wf.non_steady_state" found cached.
181217-15:49:55,220 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.fdisp" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-01_run-2/bold_confs_wf/fdisp".
181217-15:49:55,223 nipype.workflow INFO:
	 [Node] Cached "bold_confs_wf.fdisp" - collecting precomputed outputs
181217-15:49:55,224 nipype.workflow INFO:
	 [Node] "bold_confs_wf.fdisp" found cached.
181217-15:49:55,225 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.dvars" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-01_run-2/bold_confs_wf/dvars".
181217-15:49:55,228 nipype.workflow INFO:
	 [Node] Cached "bold_confs_wf.dvars" - collecting precomputed outputs
181217-15:49:55,229 nipype.workflow INFO:
	 [Node] "bold_confs_wf.dvars" found cached.
181217-15:49:55,229 nipype.wor

181217-15:49:55,522 nipype.workflow INFO:
	 [Node] Cached "bold_confs_wf.dvars" - collecting precomputed outputs
181217-15:49:55,523 nipype.workflow INFO:
	 [Node] "bold_confs_wf.dvars" found cached.
181217-15:49:55,524 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.get_first_slice" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-01_run-3/bold_confs_wf/get_first_slice".
181217-15:49:55,527 nipype.workflow INFO:
	 [Node] Cached "bold_confs_wf.get_first_slice" - collecting precomputed outputs
181217-15:49:55,528 nipype.workflow INFO:
	 [Node] "bold_confs_wf.get_first_slice" found cached.
181217-15:49:55,529 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.downsample_tmps" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-01_run-3/bold_confs_wf/downsample_tmps".
181217-15:49:55,540 nipype.workflow INFO:
	 [Node] "bold_confs_wf.downsample_tmps" found cached.
181217-15:49:55,541 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.wm_roi" in "/home/

181217-15:49:56,318 nipype.workflow INFO:
	 [Node] "bold_confs_wf.downsample_tmps" found cached.
181217-15:49:56,320 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.wm_roi" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-02_run-1/bold_confs_wf/wm_roi".
181217-15:49:56,330 nipype.workflow INFO:
	 [Node] Cached "bold_confs_wf.wm_roi" - collecting precomputed outputs
181217-15:49:56,331 nipype.workflow INFO:
	 [Node] "bold_confs_wf.wm_roi" found cached.
181217-15:49:56,332 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.wm_msk" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-02_run-1/bold_confs_wf/wm_msk".
181217-15:49:56,344 nipype.workflow INFO:
	 [Node] Cached "bold_confs_wf.wm_msk" - collecting precomputed outputs
181217-15:49:56,345 nipype.workflow INFO:
	 [Node] "bold_confs_wf.wm_msk" found cached.
181217-15:49:56,346 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.csf_roi" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-02

181217-15:49:56,680 nipype.workflow INFO:
	 [Node] "bold_confs_wf.wm_msk" found cached.
181217-15:49:56,681 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.csf_roi" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-02_run-2/bold_confs_wf/csf_roi".
181217-15:49:56,685 nipype.workflow INFO:
	 [Node] Cached "bold_confs_wf.csf_roi" - collecting precomputed outputs
181217-15:49:56,686 nipype.workflow INFO:
	 [Node] "bold_confs_wf.csf_roi" found cached.
181217-15:49:56,687 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.csf_msk" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-02_run-2/bold_confs_wf/csf_msk".
181217-15:49:56,700 nipype.workflow INFO:
	 [Node] Cached "bold_confs_wf.csf_msk" - collecting precomputed outputs
181217-15:49:56,701 nipype.workflow INFO:
	 [Node] "bold_confs_wf.csf_msk" found cached.
181217-15:49:56,702 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.merge_rois" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-

...

181217-16:27:27,920 nipype.workflow INFO:
	 [Node] Finished "bold_confs_wf.dvars".
181217-16:27:27,923 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.get_first_slice" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-04_run-3/bold_confs_wf/get_first_slice".
181217-16:27:27,931 nipype.workflow INFO:
	 [Node] Running "get_first_slice" ("nipype.interfaces.utility.wrappers.Function")
181217-16:27:47,78 nipype.workflow INFO:
	 [Node] Finished "bold_confs_wf.get_first_slice".
181217-16:27:47,79 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.downsample_tmps" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-04_run-3/bold_confs_wf/downsample_tmps".
181217-16:27:47,91 nipype.workflow INFO:
	 [Node] Setting-up "_downsample_tmps0" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-04_run-3/bold_confs_wf/downsample_tmps/mapflow/_downsample_tmps0".
181217-16:27:47,100 nipype.workflow INFO:
	 [Node] Running "_downsample_tmps0" ("nipype.interfaces.utility

181217-16:29:59,241 nipype.workflow INFO:
	 [Node] Finished "bold_confs_wf.downsample_tmps".
181217-16:29:59,242 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.wm_roi" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-04_run-3/bold_confs_wf/wm_roi".
181217-16:29:59,247 nipype.workflow INFO:
	 [Node] Running "wm_roi" ("fmriprep.interfaces.utils.TPM2ROI")
181217-16:30:00,249 nipype.workflow INFO:
	 [Node] Finished "bold_confs_wf.wm_roi".
181217-16:30:00,250 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.wm_msk" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-04_run-3/bold_confs_wf/wm_msk".
181217-16:30:00,254 nipype.workflow INFO:
	 [Node] Running "wm_msk" ("nipype.interfaces.utility.wrappers.Function")
181217-16:30:00,317 nipype.workflow INFO:
	 [Node] Finished "bold_confs_wf.wm_msk".
181217-16:30:00,318 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.csf_roi" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-04_run-3/bold_confs_

/usr/local/miniconda/lib/python3.6/site-packages/nipype/algorithms/confounds.py:1022: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  betas = np.linalg.lstsq(X, data.T)[0]


181217-16:34:47,507 nipype.workflow INFO:
	 [Node] Finished "bold_confs_wf.tcompcor".
181217-16:34:47,512 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.tpms_add_csf_wm" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-04_run-3/bold_confs_wf/tpms_add_csf_wm".
181217-16:34:47,519 nipype.workflow INFO:
	 [Node] Running "tpms_add_csf_wm" ("fmriprep.interfaces.utils.AddTPMs")
181217-16:34:47,985 nipype.workflow INFO:
	 [Node] Finished "bold_confs_wf.tpms_add_csf_wm".
181217-16:34:47,986 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.acc_roi" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-04_run-3/bold_confs_wf/acc_roi".
181217-16:34:47,991 nipype.workflow INFO:
	 [Node] Running "acc_roi" ("fmriprep.interfaces.utils.TPM2ROI")
181217-16:34:49,242 nipype.workflow INFO:
	 [Node] Finished "bold_confs_wf.acc_roi".
181217-16:34:49,243 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.acc_msk" in "/home/neuro/work_dir/workflow_folders/confounds_me_s

/usr/local/miniconda/lib/python3.6/site-packages/nitime/utils.py:980: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  complex_result = (np.issubdtype(in1.dtype, np.complex) or
/usr/local/miniconda/lib/python3.6/site-packages/nitime/utils.py:981: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  np.issubdtype(in2.dtype, np.complex))


In [ ]:
bids_dir = './data/raw/bids/'
session_id = 'me'
task_id = None
mem_gb = {'filesize': 1, 'resampled': 1, 'largemem': 1}
subject_id = 1
subject_data, layout = collect_data(bids_dir, subject_id, session_id, task_id)
fn = './data/raw/bids/sub-01/ses-me/func/sub-01_ses-me_task-stop_run-1_echo-1_bold.nii'
metadata = layout.get_metadata(fn)

# # loop over subbs
for subject_id in np.arange(1,19):
    if subject_id == 12:
        continue
    
    subject_data, layout = collect_data(bids_dir, subject_id, session_id, task_id)
    
    for run in [1,2,3]:

        # get fn for metadata. NOT the fn used to get confounds from
        fn = './data/raw/bids/sub-%s/ses-me/func/sub-%s_ses-me_task-stop_run-%d_echo-1_bold.nii' %(str(subject_id).zfill(2), str(subject_id).zfill(2),run)
        metadata = layout.get_metadata(fn)
        bold_confs_wf = init_bold_confs_wf(mem_gb=mem_gb, metadata=metadata)
        bold_confs_wf.base_dir = '/home/neuro/work_dir/workflow_folders/confounds_me_denoised_sub-%s_run-%d' %(str(subject_id).zfill(2),run)

        # # inputs
        bold = '/home/neuro/work_dir//tedana_denoising/TED.sub-%s_ses-me_task-stop_run-%d_echo-1_bold_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_preproc_hp_fwhm32/dn_ts_OC.nii' %(str(subject_id).zfill(2),run)
        bold_mask = '/home/neuro/work_dir/data/deriv/fmriprep/sub-%s/ses-me/func/sub-%s_ses-me_task-stop_run-%d_echo-1_bold_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_preproc_hp.nii.gz' %(str(subject_id).zfill(2), str(subject_id).zfill(2),run)
        bold_mask = bold_mask.replace('_preproc_hp.nii.gz', '_brainmask.nii.gz').replace('optcomb', 'echo-1')
        
        if subject_id < 5:
            movpar_file = '/home/neuro/work_dir/workflow_folders/fmriprep_wf_sub-01-04/single_subject_%s_wf/func_preproc_ses_me_task_stop_run_%d_echo_1_wf/bold_hmc_wf/normalize_motion/motion_params.txt' %(str(subject_id).zfill(2), run)
        else:
            movpar_file = '/home/neuro/work_dir/workflow_folders/fmriprep_wf/single_subject_%s_wf/func_preproc_ses_me_task_stop_run_%d_echo_1_wf/bold_hmc_wf/normalize_motion/motion_params.txt' %(str(subject_id).zfill(2), run)

        t1_mask = bold_mask #'/home/neuro/work_dir/data/deriv/fmriprep/sub-%s/anat/sub-%s_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_brainmask.nii.gz' %(str(subject_id).zfill(2), str(subject_id).zfill(2))
        t1_tpms = [
            '/home/neuro/work_dir/data/deriv/fmriprep/sub-%s/anat/sub-%s_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_class-CSF_probtissue.nii.gz'%(str(subject_id).zfill(2), str(subject_id).zfill(2)), 
            '/home/neuro/work_dir/data/deriv/fmriprep/sub-%s/anat/sub-%s_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_class-GM_probtissue.nii.gz'%(str(subject_id).zfill(2), str(subject_id).zfill(2)),
            '/home/neuro/work_dir/data/deriv/fmriprep/sub-%s/anat/sub-%s_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_class-WM_probtissue.nii.gz'%(str(subject_id).zfill(2), str(subject_id).zfill(2))
        ]

        bold_confs_wf.inputs.inputnode.bold = bold
        bold_confs_wf.inputs.inputnode.bold_mask = bold_mask
        bold_confs_wf.inputs.inputnode.movpar_file = movpar_file
        bold_confs_wf.inputs.inputnode.t1_mask = t1_mask
        bold_confs_wf.inputs.inputnode.t1_tpms = t1_tpms

        bold_confs_wf.outputs.outputnode.confounds_file = '/home/neuro/work_dir/confounds_me/sub-%s_ses-me_task-stop_run-%d_optcomb-denoised_bold_space-T1w_confounds.tsv' %(str(subject_id).zfill(2),run)
        
        bold_confs_wf.run()

In [94]:
bold_confs_wf.run()

181217-13:54:09,910 nipype.workflow INFO:
	 Workflow bold_confs_wf settings: ['check', 'execution', 'logging', 'monitoring']
181217-13:54:09,933 nipype.workflow INFO:
	 Running serially.
181217-13:54:09,935 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.add_header" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-01_run-1/bold_confs_wf/add_header".
181217-13:54:09,939 nipype.workflow INFO:
	 [Node] Cached "bold_confs_wf.add_header" - collecting precomputed outputs
181217-13:54:09,940 nipype.workflow INFO:
	 [Node] "bold_confs_wf.add_header" found cached.
181217-13:54:09,942 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.non_steady_state" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-01_run-1/bold_confs_wf/non_steady_state".
181217-13:54:09,946 nipype.workflow INFO:
	 [Node] Cached "bold_confs_wf.non_steady_state" - collecting precomputed outputs
181217-13:54:09,947 nipype.workflow INFO:
	 [Node] "bold_confs_wf.non_steady_state" found cached.
1


Saving /home/neuro/work_dir/data/deriv/fmriprep/sub-01/anat/sub-01_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_class-WM_probtissue_tmp_srccoord.nii.gz

Saving /home/neuro/work_dir/data/deriv/fmriprep/sub-01/anat/sub-01_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_class-WM_probtissue_tmp_trgcoord.nii.gz
registering /home/neuro/work_dir/data/deriv/fmriprep/sub-01/anat/sub-01_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_class-WM_probtissue.nii.gz
 to /home/neuro/work_dir/data/deriv/fmriprep/sub-01/ses-me/func/sub-01_ses-me_task-stop_run-1_echo-1_bold_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_preproc_hp_first_slice.nii.gz
antsRegistration --float 0 --collapse-output-transforms 1 --dimensionality 3 --initialize-transforms-per-stage 0 --interpolation Linear --output sub-01_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_class-WM_probtissue_tmp_syn --transform Rigid[ 0.1 ] --metric CC[ /home/neuro/work_dir/data/deriv/fmriprep/sub-01/ses-me/func/sub-0

/usr/local/miniconda/lib/python3.6/site-packages/nipype/algorithms/confounds.py:1022: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  betas = np.linalg.lstsq(X, data.T)[0]


181217-13:59:27,144 nipype.workflow INFO:
	 [Node] Finished "bold_confs_wf.tcompcor".
181217-13:59:27,146 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.tpms_add_csf_wm" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-01_run-1/bold_confs_wf/tpms_add_csf_wm".
181217-13:59:27,149 nipype.workflow INFO:
	 [Node] Outdated cache found for "bold_confs_wf.tpms_add_csf_wm".
181217-13:59:27,154 nipype.workflow INFO:
	 [Node] Running "tpms_add_csf_wm" ("fmriprep.interfaces.utils.AddTPMs")
181217-13:59:27,471 nipype.workflow INFO:
	 [Node] Finished "bold_confs_wf.tpms_add_csf_wm".
181217-13:59:27,472 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.acc_roi" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-01_run-1/bold_confs_wf/acc_roi".
181217-13:59:27,476 nipype.workflow INFO:
	 [Node] Outdated cache found for "bold_confs_wf.acc_roi".
181217-13:59:27,479 nipype.workflow INFO:
	 [Node] Running "acc_roi" ("fmriprep.interfaces.utils.TPM2ROI")
181217-13:59:28,1

/usr/local/miniconda/lib/python3.6/site-packages/matplotlib/contour.py:1173: UserWarning: No contour levels were found within the data range.
  warnings.warn("No contour levels were found"


181217-13:59:52,190 nipype.workflow INFO:
	 [Node] Finished "bold_confs_wf.rois_plot".
181217-13:59:52,192 nipype.workflow INFO:
	 [Node] Setting-up "bold_confs_wf.ds_report_bold_rois" in "/home/neuro/work_dir/workflow_folders/confounds_me_sub-01_run-1/bold_confs_wf/ds_report_bold_rois".
181217-13:59:52,197 nipype.workflow INFO:
	 [Node] Running "ds_report_bold_rois" ("fmriprep.interfaces.bids.DerivativesDataSink")
181217-13:59:52,199 nipype.workflow WARNING:
	 [Node] Error on "bold_confs_wf.ds_report_bold_rois" (/home/neuro/work_dir/workflow_folders/confounds_me_sub-01_run-1/bold_confs_wf/ds_report_bold_rois)
181217-13:59:52,201 nipype.workflow ERROR:
	 Node ds_report_bold_rois failed to run on host 0e230f85deac.
181217-13:59:52,203 nipype.workflow ERROR:
	 Saving crash info to /home/neuro/work_dir/crash-20181217-135952-neuro-ds_report_bold_rois-4e39e698-c26d-447b-8979-9f8c1a86792c.pklz
Traceback (most recent call last):
  File "/usr/local/miniconda/lib/python3.6/site-packages/nipype/

RuntimeError: Workflow did not execute cleanly. Check log for details

In [56]:
!ls /home/neuro/work_dir/data/deriv/fmriprep/sub-01/ses-me/func/sub-01_ses-me_task-stop_run-1_echo-1_bold_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_brainmask.nii.gz

sub-01_ses-me_task-stop_run-1_PAID_bold_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_preproc.nii.gz
sub-01_ses-me_task-stop_run-1_echo-1_bold_confounds.tsv
sub-01_ses-me_task-stop_run-1_echo-1_bold_space-T1w_brainmask.nii.gz
sub-01_ses-me_task-stop_run-1_echo-1_bold_space-T1w_preproc.nii.gz
sub-01_ses-me_task-stop_run-1_echo-1_bold_space-T1w_preproc_hp.nii.gz
sub-01_ses-me_task-stop_run-1_echo-1_bold_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_brainmask.nii.gz
sub-01_ses-me_task-stop_run-1_echo-1_bold_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_preproc.nii.gz
sub-01_ses-me_task-stop_run-1_echo-1_bold_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_preproc_hp.nii.gz
sub-01_ses-me_task-stop_run-1_echo-2_bold_confounds.tsv
sub-01_ses-me_task-stop_run-1_echo-2_bold_space-T1w_brainmask.nii.gz
sub-01_ses-me_task-stop_run-1_echo-2_bold_space-T1w_preproc.nii.gz
sub-01_ses-me_task-stop_run-1_echo-2_bold_space-T1w_preproc_hp.nii.gz
sub-01_ses-me_task-stop_run-1_echo-2_bold_spa

In [43]:
bold_confs_wf.outputs.outputnode


confounds_file = None

In [41]:
!ls /home/neuro/work_dir/data/deriv/fmriprep/sub-01/anat/sub-01_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_brainmask.nii.gz

sub-01_T1w_brainmask.nii.gz
sub-01_T1w_class-CSF_probtissue.nii.gz
sub-01_T1w_class-GM_probtissue.nii.gz
sub-01_T1w_class-WM_probtissue.nii.gz
sub-01_T1w_dtissue.nii.gz
sub-01_T1w_preproc.nii.gz
sub-01_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_brainmask.nii.gz
sub-01_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_class-CSF_probtissue.nii.gz
sub-01_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_class-GM_probtissue.nii.gz
sub-01_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_class-WM_probtissue.nii.gz
sub-01_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_dtissue.nii.gz
sub-01_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_preproc.nii.gz
sub-01_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_target-T1w_warp-coordmapping.nii.gz
sub-01_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_target-T1w_warp0000.mat
sub-01_T1w_space-mni_icbm152_t1_tal_nlin_asym_09a_brain_d4mm_target-T1w_warp0001.mat
sub-01_T1w_target-mni_icbm152_t1_tal_nlin_a